# 🏗️ Notebook: Einführung in strukturierte Outputs & Validierung bei LLMs

In diesem Notebook lernen wir, wie man Large Language Models (LLMs) dazu bringt, strukturierte Ausgaben zu erzeugen.

## 📚 Quellen

- [OpenAI: Validating LLM Outputs](https://platform.openai.com/docs/guides/structured-outputs?api-mode=chat)
- [Pydantic Dokumentation](https://pydantic.dev/)
- [Ollama Structured Outputs](https://ollama.com/blog/structured-outputs)

---

Viel Erfolg beim Ausprobieren und Validieren! 🤗

In [1]:
# Importiere die benötigten Bibliotheken
from pydantic import BaseModel, Field
from openai import OpenAI
from typing import List, Literal, Optional
import json

In [ ]:
LLM_URL = "http://132.199.138.16:11434/v1"
LLM_MODEL = "gemma3:4b"

In [16]:
client = OpenAI(
    base_url=LLM_URL,
    api_key="ollama",
)

Im Backend läuft ein Gemma3 Modell, das über die OpenAI-kompatible API von Ollama angesprochen wird. **Erinnerung**: Neben Ollama gibt es auch andere Anbieter, die OpenAI-kompatible APIs bereitstellen, z.B. [vLLM](https://docs.vllm.ai/en/v0.8.2/features/structured_outputs.html).

**Strukturierte Ausgaben** ermöglichen es, die Ausgabe eines Modells auf ein bestimmtes Format zu beschränken, das durch ein **JSON-Schema** definiert wird. Ollama unterstüzt aktuell Strukturierte Ausgaben für das JSON-Format, ebenso wie die Entpunkte von OpenAI selbst für deren GPT Modelle. VLLM unterstützt auch Regex, allerdings gibt es noch einige Bugs (Stand Oktober 2025).

 
Anwendungsfälle für strukturierte Ausgaben:

- Daten aus Dokumenten extrahieren
- Daten aus Bildern extrahieren
- Alle Antworten von Sprachmodellen strukturieren

### Beispiel 1: Datenmodelle definieren mit Pydantic

Im ersten Beispiel definieren wir ein Datenmodell für Haustiere und lassen das Modell die Informationen aus einem Text extrahieren.
Um strukturierte Ausgaben zu ermöglichen, definieren wir zwei Klassen mit **Pydantic**. Diese Klassen beschreiben die Struktur der erwarteten Ausgabe.

**Pydantic** ist eine Python-Bibliothek zur **Datenvalidierung und -strukturierung**.   Sie ermöglicht es, mit **Python-Klassen (Modellen)** klar definierte Datenschemata für JSON-Objekte zu erstellen, die automatisch:

- Typüberprüfungen durchführen
- Daten in das gewünschte Format konvertieren

Ein detailliertes Cheat Sheet zu Pydantic findet ihr auf GRIPS.

**Ablauf**

1. **Datenmodelle definieren**  
   Zwei Klassen werden mit `BaseModel` (von **Pydantic**) definiert:
   - `Pet`: beschreibt ein einzelnes Haustier mit Attributen wie `name`, `animal`, `age`, `color`, `favorite_toy`.
   - `PetList`: enthält eine Liste mehrerer `Pet`-Objekte.

2. **Eingabe-Prompt**  
   Der Text (`prompt`) beschreibt zwei Haustiere in natürlicher Sprache.

3. **Modellaufruf mit Strukturvorgabe**  
   `client.beta.chat.completions.parse()` ruft das Sprachmodell auf und nutzt `response_format=PetList`, um die Ausgabe **automatisch in die definierte JSON-Struktur** zu parsen.

4. **Ausgabe**  
   Das Modell gibt die erkannten Haustiere als **strukturiertes JSON-Objekt** zurück, das automatisch den Feldern von `PetList` entspricht.


In [17]:
class Pet(BaseModel):
    name: str
    animal: str
    age: int
    color: str | None
    favorite_toy: str | None


class PetList(BaseModel):
    pets: list[Pet]


prompt = '''
I have two pets.
A cat named Luna who is 5 years old and loves playing with yarn. She has grey fur.
I also have a 2 year old black cat named Loki who loves tennis balls.
'''

completion = client.chat.completions.parse(
    temperature=0,
    model=LLM_MODEL,
    messages=[
        {"role": "user", "content": prompt}
    ],
    response_format=PetList,
)

output = completion.choices[0].message.content

# Die LLM-Antwort ist im String-Format. Wir müssen sie in ein Python-Dictionary umwandeln.
output_dict = json.loads(output)
print(output_dict)

{'pets': [{'name': 'Luna', 'animal': 'cat', 'age': 5, 'color': 'grey', 'favorite_toy': 'yarn'}, {'name': 'Loki', 'animal': 'cat', 'age': 2, 'color': 'black', 'favorite_toy': 'tennis balls'}]}


### Beispiel 2: Textklassifikation

Wir können auch ein einfaches Klassifikationsschema definieren, um Texte zu kategorisieren. In diesem Beispiel klassifizieren wir kurze Texte in die Kategorien "positive", "negative" oder "neutral".

In [5]:
class SentimentResult(BaseModel):
    sentiment: Literal["positive", "neutral", "negative"]

prompt = f"Classify the sentiment of this text as positive, neutral, or negative:\n\nI love programming in Python!"

completion = client.chat.completions.parse(
    temperature=0,
    model=LLM_MODEL,
    messages=[
        {"role": "user", "content": prompt}
    ],
    response_format=SentimentResult,
)

output = completion.choices[0].message.content

# Die LLM-Antwort ist im String-Format. Wir müssen sie in ein Python-Dictionary umwandeln.
output_dict = json.loads(output)
print(f"Output Dict: {output_dict}")

# Geben wir nur die Stimmung aus
print(f"Stimmung: {output_dict['sentiment']}")

Output Dict: {'sentiment': 'positive'}
Stimmung: positive


### Beispiel 3: JSON Schema im Prompt

Wir können dem LLM auch das json schema direkt im Prompt übergeben. `json.dumps(json_schema, indent=2)` wandelt das Python-Dictionary in einen JSON-String um, der im Prompt verwendet werden kann. `indent=2` sorgt für eine lesbare Formatierung mit Einrückungen. 2 bedeutet, dass jede Ebene im JSON um 2 Leerzeichen eingerückt wird.

In [6]:
# Definiere eine Pydantic-Klasse für einen einzelnen Softwareentwickler
class Developer(BaseModel):
    name: str  # Name des Entwicklers
    programmiersprache: Literal["python", "java", "javascript", "csharp", "cpp", "go", "rust", "php"]  # Hauptprogrammiersprache
    erfahrung_jahre: int = Field(ge=0, le=50)  # Berufserfahrung in Jahren (0-50)
    spezialisierung: Literal["frontend", "backend", "fullstack", "data_science", "devops", "mobile"]  # Spezialisierungsbereich

# Definiere eine Pydantic-Klasse für eine Liste von genau 4 Entwicklern
class DeveloperList(BaseModel):
    entwickler: List[Developer] = Field(min_length=2, max_length=2)  # Genau 4 Developer-Objekte

# Erzeuge das JSON-Schema aus der DeveloperList-Klasse
json_schema = DeveloperList.model_json_schema()

prompt = f"""
Generate a JSON array of exactly 2 software developers with the following fields:
- name: string
- programmiersprache: one of ["python", "java", "javascript", "csharp", "cpp", "go", "rust", "php"]
- erfahrung_jahre: integer between 0 and 50
- spezialisierung: one of ["frontend", "backend", "fullstack", "data_science", "devops", "mobile"]      
Make sure the output strictly adheres to this JSON schema:
{json.dumps(json_schema, indent=2)}
"""

completion = client.chat.completions.parse(
    temperature=0,
    model=LLM_MODEL,
    messages=[
        {"role": "user", "content": prompt}
    ],
    response_format=DeveloperList,
)

output = completion.choices[0].message.content

# Die LLM-Antwort ist im String-Format. Wir müssen sie in ein Python-Dictionary umwandeln.
output_dict = json.loads(output)
print(output_dict)

{'entwickler': [{'name': 'Alice Müller', 'programmiersprache': 'python', 'erfahrung_jahre': 8, 'spezialisierung': 'backend'}, {'name': 'Bob Schmidt', 'programmiersprache': 'javascript', 'erfahrung_jahre': 2, 'spezialisierung': 'frontend'}]}


### Übungsaufgabe 1

Um die Validierung von JSON-Ausgaben zu üben, erstelle ein Prompt, das das LLM anweist, eine strukturierte Antwort im JSON-Format zu liefern.

---

#### 🎯 Aufgabe

Definiere ein einfaches JSON-Schema um Dummy-Daten zu **Musikern** zu generieren.

#### 🎤 Interpret-Schema

| Feld     | Beschreibung                                                |
| -------- | ----------------------------------------------------------- |
| `name`   | Name des Interpreten                                        |
| `age`    | Alter (zwischen 18 und 100)                                 |
| `genre`  | Musikrichtung aus: **Rock**, **Pop**, **Jazz**, **Klassik** |
| `albums` | Liste von Alben                                             |

#### 💿 Album-Schema

| Feld           | Beschreibung                              |
| -------------- | ----------------------------------------- |
| `title`        | Album-Titel                               |
| `release_year` | Erscheinungsjahr (zwischen 1900 und 2023) |
| `tracks`       | Liste von Songtiteln                      |

---

#### 💡 Tipp

Nutze Pydantic-Klassen mit `Field()`-Validierungen und das `Literal`-Typ-System für die Genre-Auswahl!


<details>
<summary><b>Lösung anzeigen</b></summary>

```python
# Definiere eine Pydantic-Klasse für ein Album
class Album(BaseModel):
    title: str
    release_year: int = Field(ge=1900, le=2023)
    tracks: List[str]

# Definiere eine Pydantic-Klasse für einen Musiker
class Musician(BaseModel):
    name: str
    age: int = Field(ge=18, le=100)
    genre: Literal["Rock", "Pop", "Jazz", "Klassik"]
    albums: List[Album]

# Definiere eine Pydantic-Klasse für eine Liste von Musikern
class MusicianList(BaseModel):
    musicians: List[Musician] = Field(min_length=2, max_length=2)

# Erzeuge das JSON-Schema aus der MusicianList-Klasse
json_schema = MusicianList.model_json_schema()

prompt = f"""
Generiere eine Liste von 2 Musikern mit den folgenden Feldern:
- name: string
- age: integer zwischen 18 und 100
- genre: eines von ["Rock", "Pop", "Jazz", "Klassik"]
- albums: Liste von Alben, jedes Album hat:
  - title: string
  - release_year: integer zwischen 1900 und 2023
  - tracks: Liste von Songtiteln (strings)

Gib die Daten als JSON zurück, das streng diesem Schema entspricht:
{json.dumps(json_schema, indent=2)}
"""

completion = client.beta.chat.completions.parse(
    temperature=0,
    model=LLM_MODEL,
    messages=[
        {"role": "user", "content": prompt}
    ],
    response_format=MusicianList,
)

output = completion.choices[0].message.content

# Die LLM-Antwort ist im String-Format. Wir müssen sie in ein Python-Dictionary umwandeln.
output_dict = json.loads(output)
print(json.dumps(output_dict, indent=2, ensure_ascii=False))
```

</details>

In [7]:
### Dein Code...

### Beispiel 4: Guided JSON für strukturierte Extraktion von Informationen aus Dokumenten
Beispiel für strukturierte Extraktion von Informationen aus einem Bild.

Beispiel:

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ0s5B5TIgNtd8NBG31BBu2v1cCxIZi3AEE2g&s" alt="Structured Extraction Example" width="100">

In [8]:
# Laden wir das Bild und kodieren es in Base64. Base64 ist ein Textformat, das binäre Daten (wie Bilder) in eine Textdarstellung umwandelt.
import base64
with open("beispiel_bild.png", "rb") as image_file:
    # Datei in Base64 umwandeln
    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")

# Als Data-URL formatieren. Data-URLs ermöglichen es, Bilder direkt in HTML oder JSON einzubetten.
data_url = f"data:image/png;base64,{encoded_string}"

In [9]:
# Beginnen wir zunächst ohne eine JSON Validierung, um die Ausgabe zu testen.
# Das LLM soll zunächst in der Lage sein, ein Bild zu analysieren und eine Antwort zu generieren.

image_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ0s5B5TIgNtd8NBG31BBu2v1cCxIZi3AEE2g&s"

prompt = "Was siehst du auf diesem Bild?"
completion = client.chat.completions.parse(
    temperature=0,
    model=LLM_MODEL,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": data_url
                },
            ],
        }
    ]
)

print(completion.choices[0].message.content)

Auf dem Bild sehe ich das berühmte Gemälde der Mona Lisa, auch bekannt als La Gioconda. Es ist ein Porträt einer Frau, vermutlich Lisa Gherardini, gemalt von Leonardo da Vinci. 

Hier sind einige Details, die ich sehe:

*   **Die Frau:** Sie hat ein sanftes Lächeln und einen nachdenklichen Blick.
*   **Kleidung:** Sie trägt dunkle Kleidung mit einem goldenen Faden.
*   **Haltung:** Ihre Hände sind gefaltet im Schoß.
*   **Hintergrund:** Es gibt eine verschwommene Landschaft im Hintergrund.

Es ist eines der bekanntesten und meistbesuchten Kunstwerke der Welt.


### 4. Guided JSON für strukturierte Extraktion von Informationen aus Dokumenten

Nun werden wir ein komplexeres Beispiel betrachten: die strukturierte Extraktion von Informationen aus einem Rechnungsdokument.

#### Ziel der Übung

Wir möchten ein Rechnungsdokument analysieren und die wichtigsten Informationen automatisch extrahieren:
- **Liste der Produkte** mit Details
- **Gesamtsumme** der Rechnung

#### 🖼️ Das Dokument

So sieht das zu analysierende Rechnungsdokument aus:

<img src="https://www.buchhaltungsbutler.de/wp-content/uploads/rechnungsvorlage11-e1692972330525.png" alt="Rechnung Beispiel" width="400">

In [10]:
with open("beispiel_rechnung.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
data_url = f"data:image/png;base64,{encoded_string}"

In [11]:
class InvoiceItem(BaseModel):
    beschreibung: str = Field(min_length=1, max_length=100)
    anzahl: int = Field(ge=0, le=5)
    einzelpreis_in_euro: int = Field(ge=0, le=1000)

class Invoice(BaseModel):
    gesamtbetrag: float = Field(gt=0)
    artikel: List[InvoiceItem] = Field(min_length=1)

In [12]:
prompt = "Analysiere die Rechnung auf dem Bild und gib mir die Details als JSON. Die Anzahl an Artikeln entspricht der Anzahl an JSON-Objekten in der Liste: "
completion = client.chat.completions.parse(
    temperature=0,
    model=LLM_MODEL,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": data_url
                },
            ],
        }
    ],
    response_format=Invoice,
)

print(completion.choices[0].message.content)

{"gesamtbetrag": 35.70, "artikel": [
    {
        "beschreibung": "Musterprodukt",
        "anzahl": 5,
        "einzelpreis_in_euro": 300
    },
    {
        "beschreibung": "Musterprodukt",
        "anzahl": 3,
        "einzelpreis_in_euro": 500
    }
  ]
}


### Übungsaufgabe 2

Jetzt bist du dran! Du möchtest automatisiert Informationen aus Rechnungen extrahieren. 

Beispiel:

<img src="https://images.t-online.de/2021/06/89589144v1/0x0:768x1024/fit-in/1366x0/image.jpg" alt="Rechnung Beispiel" width="400">

Das LLM soll folgende strukturierte Informationen im JSON-Format zurückgeben:

* Verkäuferinformationen (alle Keys optional):
  - Name
  - Adresse

* Summe der Rechnung
* Liste der Produkte, jeweils mit:
  - Produkt-ID
  - Produktname
  - Preis in Euro
  - Steuergruppe
  - Gesamtpreis

<details>
<summary><b>Lösung anzeigen</b></summary>

```python
# Definiere eine Pydantic-Klasse für Verkäuferinformationen
class SellerInfo(BaseModel):
    name: Optional[str] = None
    address: Optional[str] = None

# Definiere eine Pydantic-Klasse für ein Produkt
class Product(BaseModel):
    product_id: str
    product_name: str
    price_euro: float = Field(ge=0)
    tax_group: str
    total_price: float = Field(ge=0)

# Definiere eine Pydantic-Klasse für die Rechnung
class Invoice(BaseModel):
    seller: SellerInfo
    total_amount: float = Field(ge=0)
    products: List[Product] = Field(min_length=1)

# Laden wir das Bild und kodieren es in Base64
with open("beispiel_kassenzettel.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
data_url = f"data:image/png;base64,{encoded_string}"

prompt = "Analysiere die Rechnung auf dem Bild und extrahiere die Verkäuferinformationen, die Gesamtsumme und die Liste der Produkte als JSON."

completion = client.beta.chat.completions.parse(
    temperature=0,
    model=LLM_MODEL,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": data_url
                },
            ],
        }
    ],
    response_format=Invoice,
)

output = completion.choices[0].message.content

# Die LLM-Antwort ist im String-Format. Wir müssen sie in ein Python-Dictionary umwandeln.
output_dict = json.loads(output)
print(json.dumps(output_dict, indent=2, ensure_ascii=False))
```

</details>

In [13]:
# Hier steht dein Code...